In [2]:
#Downloading the data

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_a020f6394b8b45d3ab934ea8c30064c3 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='1wz9PKHXdqo5CfvhES1UBmJo_QbQxRvNg8n2_kp6BLg3',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_a020f6394b8b45d3ab934ea8c30064c3.get_object(Bucket='martatorontox27sneighborhoods-donotdelete-pr-yuoe5snms5lodj',Key='Toronto_Neighborhoods.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_0 = pd.read_excel(body)
df_data_0.head()


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [3]:
df_data_0.shape

(287, 3)

In [4]:
#Excluding the Not Assigned Boroughs

df_toronto = df_data_0[df_data_0.Borough != 'Not assigned']
df_toronto.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Not assigned
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [5]:
df_toronto.shape

(210, 3)

In [6]:
# Finding the Not assigned Neighbourhoods

df_toronto.loc[df_toronto.Neighbourhood == 'Not assigned']

,Postcode,Borough,Neighbourhood
9,M9A,Queen's Park,Not assigned


In [7]:
# Assigning the Borough's name to the Neighbourhood

df_toronto.at[9, 'Neighbourhood'] = "Queen's Park"

In [8]:
df_toronto.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [9]:
# Grouping the data by Postcode

df_toronto2 = df_toronto.groupby('Postcode').agg({'Borough':'first','Neighbourhood':', '.join})
df_toronto2.head(10)

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"


In [10]:
df_toronto2.shape

(103, 2)

In [15]:

body = client_a020f6394b8b45d3ab934ea8c30064c3.get_object(Bucket='martatorontox27sneighborhoods-donotdelete-pr-yuoe5snms5lodj',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_data_0 = pd.read_csv(body)
df_data_0.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [44]:
df_data_0.shape

(103, 4)

In [56]:
df_toronto3 = pd.merge(df_data_0, df_toronto2, left_on='Postal Code',  right_on='Postcode', how='left')


In [57]:
df_toronto3.head()

,Postal Code,Latitude,Longitude,Borough_x,Borough_y,Neighbourhood
0,M1B,43.806686,-79.194353,NaN,Scarborough,"Rouge, Malvern"
1,M1C,43.784535,-79.160497,NaN,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,43.763573,-79.188711,NaN,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,NaN,Scarborough,Woburn
4,M1H,43.773136,-79.239476,NaN,Scarborough,Cedarbrae


In [58]:
df_toronto4= df_toronto3.drop('Borough_x', 1)
df_toronto4.head()

,Postal Code,Latitude,Longitude,Borough_y,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Rouge, Malvern"
1,M1C,43.784535,-79.160497,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [59]:
df_toronto5 = df_toronto4[['Postal Code','Borough_y', 'Neighbourhood', 'Latitude','Longitude']]
df_toronto5.head()

,Postal Code,Borough_y,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [60]:
df_toronto6 = df_toronto5.rename(columns={'Borough_y': 'Borough'})
df_toronto6.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [61]:
df_toronto6.shape

(103, 5)